# GDML Explorer

Author: [Sam Eriksen](mailto:sam.eriksen@bristol.ac.uk)

## Purpose
To work out how to explore GDML files to convert to trees for GPU simulations.

##### Unusual Libraries Needed
* lxml

### GDML Playing

Start by understanding the structure and using the GDML structure.
Eventually I'll turn this into a class (similar to opticks approach)

A good place to start is the [lxml tutorial](https://lxml.de/tutorial.html)

In [55]:
from lxml import etree
import numpy as np

In [56]:
#
gdml_file = "/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml"
gdml_e = etree.parse(gdml_file).getroot()
print('gdml element:',gdml_e)

# Get info of element
print('\nGDML structure\n--------------')
for child in gdml_e:
    print(child.tag)

gdml element: <Element gdml at 0x2aaad0c6e2c8>

GDML structure
--------------
define
materials
solids
structure
userinfo
setup


## Learning Navigation 
Write a class to extract different information

In [75]:
from lxml import etree
class GDML():
    
    def __init__(self, path):
        
        self.path = path
        self.fulletree = etree.parse(self.path).getroot()
        self.materials = self._findallnames("materials/material")
        self.nmaterials = len(self.materials)
        self.structures = self._findallnames("structure/*")
        self.nstructures = len(self.structures)
        self.world = self._findWorld()
        
    def _findallnames(self, tofind):
        allthings = self.fulletree.findall(tofind)
        valuelist = []
        for value in allthings:
            attribute = value.attrib
            valuelist.append(attribute['name'])
        return valuelist
        
    def _findWorld(self):
        world_setup = self.fulletree.findall("setup/world")
        world = world_setup[0].attrib['ref']
        return(world)


In [76]:
a = GDML("/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml")

In [77]:
a.path

'/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml'

In [79]:
a.materials

['quartz0xe969e0',
 'pcv0xe96140',
 'ss3160xe9bba0',
 'elastomer0xe98e50',
 'steel0xe9b020',
 'lzHVConduitInnerCone0xeb14a0',
 'polyFoam0xe94670',
 'aluminum0xe7e8f0',
 'nicomic0xe99780',
 'inconel0xe9a370',
 'peek0xe931f0',
 'gasXe0xe86fd0',
 'teflon0xe9d950',
 'titanium0xe9da70',
 'sapphire0xe98230',
 'vacuum0xea27f0',
 'Kovar0xeb74a0',
 'alumina0xe98550',
 'liquidXe0xe91d30',
 'dopedLABGd0xe8db20',
 'lzSpecRef0xea16b0',
 'acrylic0xe760f0',
 'lzHVConduitUmbilical0xeb2de0',
 'lzPMTCablingConduit0xeab3b0',
 'lzThermosyphonConduit0xeac950',
 'lzPMTBottomConduit0xeae210',
 'water0xea2ed0',
 'tyvek6000xea1f50']

# More Complex Operations

Before turning into a binary tree, have to extract more information out and link it.

Firstly, create a class similar to the one above which handles the elements from the etree in a better way.
These elements can be a number of different things including, eg a Material or a Volume

In [240]:
class GDMLElement(object):

    def __init__(self, elem, g=None):
        self.elem = elem
        self.g = g
        self.name = (lambda self:self.elem.attrib.get('name',None))
        self.typ = (lambda self:self.__class__.__name__)
        self.gidx = (lambda self:"[%d]" % self.idx if hasattr(self, 'idx') else '[]' )
        self.name = lambda self:self.elem.attrib.get('name',None)
        
    def _findall(self, expr):
        wrap_ = lambda e:self.g.kls.get(e.tag,G)(e,self.g)
        fa = map(wrap_, self.elem.findall(expr) )
        kln = self.__class__.__name__
        name = self.name
        return fa
    
    def att(self, name, default=None, typ=None):
        assert typ is not None
        v = self.elem.attrib.get(name, default)
        return typ(v)

Next need to handle each of the types of things which will be in the GDML file.
A class is needed for each of these

In [241]:
# Types of things in the
kls = {
        "material":Material
    }
import collections

In [242]:
class odict(collections.OrderedDict):
    """
    Used for GDML collections of materials, solids and volumes which are 
    always keyed by name.  Call method for convenient indexed access. 
    """
    def __call__(self, iarg):
        return self.items()[iarg][1]

In [243]:
class Material(GDMLElement):
    state = property(lambda self:self.att('state', '', typ=str ))
    def __repr__(self):
        return "%s %s %s %s" % (self.gidx, self.typ, self.name, self.state )

In [261]:
parse_ = lambda _:etree.parse(os.path.expandvars(_)).getroot()
tostring_ = lambda _:etree.tostring(_)
import os
class GDML(GDMLElement):
# Types of things in the
    kls = {
            "material":Material
        }
    @classmethod
    def parse(cls, path="/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml"):
        gg = parse_(path)
        a = GDMLElement(gg)
        wgg = cls.wrap(gg, path=path)
        return wgg 

    @classmethod
    def fromstring(cls, st ):
        gg = ET.fromstring(st) 
        wgg = cls.wrap(gg)
        return wgg 

    @classmethod
    def wrap(cls, gdml, path=None):
        gg = cls(gdml)
        gg.g = gg
        gg.path = path
        gg.string = tostring_(gdml)
        gg.init()
        return gg 

    def find_materials(self, prefix="/dd/Materials/Acrylic"):
        return self.find_by_prefix(self.materials, prefix)

    
    def find_by_prefix(self, d, prefix):
        return filter(lambda v:v.name.startswith(prefix), d.values())
    
    def find_materials(self, prefix="/dd/Materials/Acrylic"):
        return self.find_by_prefix(self.materials, prefix)
    
    
    world = property(lambda self:self.volumes[self.worldvol])

    def init(self):
    
        self.materials = odict()
        for i, e in enumerate(self._findall("materials/material")):
            e.idx = i
            self.materials[e.name] = e 
        pass

In [262]:
a = GDML.parse()

In [267]:
a.find_materials("tyvek6000xea1f50")